# Tutorial for AFMpy.SimAFM

## Imports

In [ ]:
# Standard library imports
import json
import logging
from pathlib import Path

# Third party imports
import MDAnalysis as MDA
import matplotlib.pyplot as plt

# AFMpy imports
from AFMpy import SimAFM, Stack, Plotting, Utilities

## Logging Configuration

In [ ]:
# Load the preconfigured logging settings
with open('logs/SimAFM_Tutorial_LoggingConfig.json', 'r') as f:
    LOGGING_CONFIG = json.load(f)

# Set up the logging configuration
logging.config.dictConfig(LOGGING_CONFIG)

## Matplotlib Configuration

In [ ]:
# Configure the rcParams for the plots
Plotting.configure_formatting()

## Load the Trajectory

In [ ]:
# Define the paths to the PSF and DCD files
PSF_PATH = '../common/MD/Example_SecYEG.psf'
DCD_PATH = '../common/MD/Example_AlignedSecYEG.dcd'

# Load the PSF and DCD files using MDAnalysis
universe = MDA.Universe(PSF_PATH, DCD_PATH)

## Set the Scanning Parameters

In [ ]:
# Tip shape parameters
tip_radius = 20
tip_theta = 18

# Scanning boundaries and shape
boundaries = ((-64,64),(-64,64))
scan_shape = (64,64)

# Atom Selections. Determine which atoms to scan, and which to consider the background.
# In this case, we scan the protein atoms, considering the heads of the lipids as the background.
protein_selection = 'protein'
membrane_selection = 'segid MEMB'
head_selection = 'name P'

# Choose the VDW radius mapping. In this case, we use the All Atom mapping.
vdw_mapping = SimAFM.AA_VDW_Dict()

## Run the AFM Simulation

In [ ]:
# Create the grid of points to scan.
grid = SimAFM.make_grid(boundaries, scan_shape)

simulated_scans = SimAFM.simulate_AFM2D_stack(universe = universe,
                                              atom_selection = protein_selection,
                                              memb_selection = membrane_selection,
                                              head_selection = head_selection,
                                              grid = grid,
                                              tip_radius = tip_radius,
                                              tip_theta = tip_theta,
                                              vdw_dict = vdw_mapping)

## Create the Stack Object

In [ ]:
# Create a dictionary to save the metadata of the scan.
mdetadata = {
    'aligned': True,
    'alignment_method': 'Transmembrane Backbone RMSD Minimization',
    'side': 'Periplasmic',
    'normalized': False,
    'PDB Code': '3DIN',
    'Membrane Composition': 'POPE',
    'Trajectory File': 'Example_AlignedSecYEG.dcd',
    'PSF File': 'Example_SecYEG.psf',
    'Tip Radius': tip_radius,
    'Tip Theta': tip_theta
}

# Calculate the resolution from the grid
resolution = (boundaries[0][1] - boundaries[0][0]) / scan_shape[0]

# Create the stack object
stack = Stack.Stack(simulated_scans, resolution = resolution, metadata = mdetadata)

## Plot the Simulated AFM image.

In [ ]:
# Plot the first scan from the stack
fig, ax = plt.subplots(figsize = (6, 6))
ax.axis('off')
ax.imshow(stack.stack[0], cmap = Plotting.LAFMcmap)

plt.show()

## Save the Stack Object to a file.

In [ ]:
# Set the paths for the private and public keys we will use to digitally sign the stack.
PRIVATE_KEY_PATH = Path('keys/Tutorial_Private.pem')
PUBLIC_KEY_PATH = Path('keys/Tutorial_Public.pub')

# If the keys do not exist, generate them.
if not PRIVATE_KEY_PATH.exists() or not PUBLIC_KEY_PATH.exists():
    Utilities.generate_keys(PRIVATE_KEY_PATH, PUBLIC_KEY_PATH)

# Save the stack and digitally sign it with the private key.
stack.save_compressed_pickle('output/SecYEG_TutorialStack.xz', private_key_filepath = PRIVATE_KEY_PATH)